# Reflection

In the context of LLM agent building, reflection refers to the process of
prompting an LLM to observe its past steps (along with potential observations
from tools/the environment) to assess the quality of the chosen actions. This is
then used downstream for things like re-planning, search, or evaluation.

![Reflection](./img/reflection.png)

This notebook demonstrates a very simple form of reflection in LangGraph.


#### Prerequisites

We will be using a basic agent with a search tool here.


## Setup

### Load env vars

Add a `.env` variable in the root of the repo folder with your variables.


In [2]:
import "dotenv/config";


## Generate

For our example, we will create a "5 paragraph essay" generator. First, create
the generator:


In [3]:
import { ChatFireworks } from "@langchain/community/chat_models/fireworks";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are an essay assistant tasked with writing excellent 5-paragraph essays.
Generate the best essay possible for the user's request.  
If the user provides critique, respond with a revised version of your previous attempts.`,
  ],
  new MessagesPlaceholder("messages"),
]);
const llm = new ChatFireworks({
  model: "accounts/fireworks/models/firefunction-v2",
  temperature: 0,
  modelKwargs: {
    max_tokens: 32768,
  },
});
const essayGenerationChain = prompt.pipe(llm);


In [4]:
import { AIMessage, BaseMessage, HumanMessage } from "@langchain/core/messages";

let essay = "";
const request = new HumanMessage({
  content:
    "Write an essay on why the little prince is relevant in modern childhood",
});

for await (
  const chunk of await essayGenerationChain.stream({
    messages: [request],
  })
) {
  console.log(chunk.content);
  essay += chunk.content;
}


The Little
 Prince, a
 novella written
 by Antoine de
 Saint-Ex
upéry in
 1943
, has been
 a beloved classic
 for generations of
 children and adults
 alike. Despite
 being written over
 80 years
 ago,
 the story
 remains remarkably
 relevant to modern
 childhood. This
 essay will explore
 the ways in
 which The Little
 Prince continues to
 resonate with
 children today,
 highlighting its timeless
 themes, rel
atable characters,
 and poignant commentary
 on the
 human experience.


One of the
 primary reasons The
 Little Prince remains
 relevant is its
 exploration of universal
 themes that transcend
 time and culture
. The story
 delves into
 complex emotions such
 as loneliness,
 friendship, and
 the importance of
 human connection.
 These themes are
 just as relevant
 today as
 they were
 when the
 book was first
 published. Children
 today face many
 of the same
 challenges as the
 Little Prince,
 including feeling isolated
 and struggling to
 form meaningful relationships
. The n

### Reflect


In [5]:
const reflectionPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a teacher grading an essay submission.
Generate critique and recommendations for the user's submission.
Provide detailed recommendations, including requests for length, depth, style, etc.`,
  ],
  new MessagesPlaceholder("messages"),
]);
const reflect = reflectionPrompt.pipe(llm);


In [6]:
let reflection = "";

for await (
  const chunk of await reflect.stream({
    messages: [request, new HumanMessage({ content: essay })],
  })
) {
  console.log(chunk.content);
  reflection += chunk.content;
}


The essay
 provides a
 good overview of
 the relevance of
 The Little Prince
 in modern childhood
. However,
 there are some
 areas that need
 improvement. Firstly
, the introduction
 could be stronger
. Instead of
 simply stating that
 the book is
 a classic,
 try to provide
 a more nuanced
 explanation of its
 enduring popularity.
 Additionally, the
 body paragraphs could
 be more detailed
 and provide more
 specific examples from
 the text to
 support the arguments
.

In terms
 of style,
 the writing is
 clear and concise
, but could
 benefit from more
 varied sentence structures
 and vocabulary.
 The use of
 transitions between paragraphs
 could also be
 improved to create
 a
 more cohesive
 flow.


One area that
 could be explored
 further is the
 way in which
 The Little Prince
 reflects the experiences
 of modern children
. While the
 essay touches on
 this, it
 could be developed
 more fully to
 provide
 a more
 nuanced understanding
 of how the
 book continues to
 resonate wi

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number
of steps, or use an LLM (or other check) to decide when the finished product is
good enough.


In [7]:
let stream = await essayGenerationChain.stream({
  messages: [
    request,
    new AIMessage({ content: essay }),
    new HumanMessage({ content: reflection }),
  ],
});
for await (const chunk of stream) {
  console.log(chunk.content);
}



Here is
 a revised
 version of
 the essay that
 addresses the areas
 mentioned above:


The Little Prince
, a nov
ella written by
 Antoine de Saint
-Exup
éry in 
1943,
 has captivated
 the hearts of
 readers of all
 ages with its
 poignant and timeless
 tale of friendship
, love,
 and the human
 experience. One
 of the primary
 reasons for its enduring
 popularity is its
 ability to tap
 into the universal
 emotions and experiences
 that define childhood
. The story
's exploration of
 complex themes such
 as loneliness,
 friendship, and
 the importance of
 human connection reson
ates deeply with
 children today,
 who face many
 of the same
 challenges as the
 Little Prince.


One of the
 most relatable
 aspects of
 The Little
 Prince is
 its
 exploration of
 the complexities
 of human relationships
. The Little
 Prince's journey
 to different planets
, where he
 encounters various strange
 characters,
 serves as
 a powerful metaphor
 for the challenges
 of forming meaningful
 connecti

## Define graph

Now that we've shown each step in isolation, we can wire it up in a graph.


In [8]:
import { END, MemorySaver, StateGraph, START, Annotation } from "@langchain/langgraph";

// Define the top-level State interface
const State = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  })
})

const generationNode = async (state: typeof State.State) => {
  const { messages } = state;
  return {
    messages: [await essayGenerationChain.invoke({ messages })],
  };
};

const reflectionNode = async (state: typeof State.State) => {
  const { messages } = state;
  // Other messages we need to adjust
  const clsMap: { [key: string]: new (content: string) => BaseMessage } = {
    ai: HumanMessage,
    human: AIMessage,
  };
  // First message is the original user request. We hold it the same for all nodes
  const translated = [
    messages[0],
    ...messages
      .slice(1)
      .map((msg) => new clsMap[msg._getType()](msg.content.toString())),
  ];
  const res = await reflect.invoke({ messages: translated });
  // We treat the output of this as human feedback for the generator
  return {
    messages: [new HumanMessage({ content: res.content })],
  };
};

// Define the graph
const workflow = new StateGraph(State)
  .addNode("generate", generationNode)
  .addNode("reflect", reflectionNode)
  .addEdge(START, "generate");

const shouldContinue = (state: typeof State.State) => {
  const { messages } = state;
  if (messages.length > 6) {
    // End state after 3 iterations
    return END;
  }
  return "reflect";
};

workflow
  .addConditionalEdges("generate", shouldContinue)
  .addEdge("reflect", "generate");

const app = workflow.compile({ checkpointer: new MemorySaver() });

In [10]:
const checkpointConfig = { configurable: { thread_id: "my-thread" } };

stream = await app.stream(
  {
    messages: [
      new HumanMessage({
        content:
          "Generate an essay on the topicality of The Little Prince and its message in modern life",
      }),
    ]
  },
  checkpointConfig,
);

for await (const event of stream) {
  for (const [key, _value] of Object.entries(event)) {
    console.log(`Event: ${key}`);
    // Uncomment to see the result of each step.
    // console.log(value.map((msg) => msg.content).join("\n"));
    console.log("\n------\n");
  }
}

Event: generate

------

Event: reflect

------

Event: generate

------

Event: reflect

------

Event: generate

------

Event: reflect

------

Event: generate

------



In [13]:
const snapshot = await app.getState(checkpointConfig);
console.log(
  snapshot.values.messages
    .map((msg: BaseMessage) => msg.content)
    .join("\n\n\n------------------\n\n\n"),
);

Generate an essay on the topicality of The Little Prince and its message in modern life


------------------


The Little Prince, a novella written by Antoine de Saint-Exupéry in 1943, has remained a timeless classic, captivating readers of all ages with its poignant and thought-provoking themes. Despite being written over seven decades ago, the story's message continues to resonate with modern society, making it a topical and relevant work of literature.

One of the primary reasons for The Little Prince's enduring popularity is its exploration of the human condition. The novella delves into the complexities of adult relationships, highlighting the superficiality and materialism that often characterize them. The Little Prince's encounters with various strange characters on different planets serve as a commentary on the flaws of modern society, where people often prioritize wealth, power, and status over genuine human connections. This critique of modern society remains pertinent today,

> #### See the LangSmith trace [here](https://smith.langchain.com/public/9bf804f7-1df8-46ef-9b40-23b3d3c97756/r)